<a href="https://colab.research.google.com/github/ByungjunKim/munzip/blob/main/%95%9C%EA%B5%AD%EA%B3%A0%EC%A0%84%EC%A2%85%ED%95%A9DB_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 한국고전종합DB API 활용하기
API : https://db.itkc.or.kr/etc/openAPI?type=page1

In [ ]:
import requests
import pandas as pd
import lxml
import xml.etree.ElementTree as ET
from tqdm import tqdm
import numpy as np
import re

### 고전원문에서 '본문검색'
예시 : 한국문집총간(본문) & '我國'  
https://db.itkc.or.kr/openapi/search?secId=MO_BD&keyword=%E6%88%91%E5%9C%8B&start=0&rows=100

In [ ]:
url = 'https://db.itkc.or.kr/openapi/search?secId=MO_BD&keyword=%E6%88%91%E5%9C%8B&start=0&rows=100'

In [ ]:
req_str = requests.get(url).text

In [ ]:
# 검색자료 총검색건수(totalCount)
totalCount = ET.fromstring(req_str).findall('.//header//field')[4].text
totalCount = int(totalCount)
totalCount

In [ ]:
xtree = ET.fromstring(req_str).findall('.//result//doc')

In [ ]:
row_list = []
for nodes in xtree:
  field = {}
  for row in nodes.findall('field'):
    field[row.attrib['name']] = row.text
  row_list.append(field)

In [ ]:
df = pd.DataFrame(row_list)
df

In [ ]:
df.columns

In [ ]:
df['검색필드']

### 원하는 조건으로 자동 수집하기
조건확인 : https://db.itkc.or.kr/etc/openAPI?type=page2  
상세검색 : https://db.itkc.or.kr/dir/detailSearchPop

In [ ]:
secld = 'MO_BD' # 한국문집총간 본문
start = 0 # 검색자료 요청 시작위치
rows = 1000 # 검색자료 요청건수
keyword = '(我國 or 吾國)' # 검색어

url = f'https://db.itkc.or.kr/openapi/search?secId={secld}&keyword={keyword}&start={start}&rows={rows}'

In [ ]:
req_str = requests.get(url).text

In [ ]:
# 검색자료 총검색건수(totalCount)
totalCount = ET.fromstring(req_str).findall('.//header//field')[4].text
totalCount = int(totalCount)
totalCount

In [ ]:
# 1000개씩 수집을 반복할 횟수
nums = totalCount // rows +1
nums

In [ ]:
row_list = []
for num in tqdm(range(nums)): # 1000개 단위로 start 변수를 바꿔가며 수집
  # url 설정
  url = f'https://db.itkc.or.kr/openapi/search?secId={secld}&keyword={keyword}&start={start+num*1000}&rows={rows}'

  # xml 요청
  req_str = requests.get(url).text
  xtree = ET.fromstring(req_str).findall('.//result//doc')

  # xml parse
  for nodes in xtree:
    field = {}
    for row in nodes.findall('field'):
      field[row.attrib['name']] = row.text
    row_list.append(field)

In [ ]:
df = pd.DataFrame(row_list)
df

##### 검색어가 포함된 구(phrase)만 추출하기

In [ ]:
df['검색필드'][0] # 검색어는 따로 표기됨 : <em class="hl1">검색어</em>

In [ ]:
# 고리표('。') 기준으로 리스트 만들기
phrase = df['검색필드'][7447].split('。')
phrase

In [ ]:
# 리스트에서 검색어가 들어간 요소(element)만 뽑아내기
[e for e in phrase if '</em>' in e]

In [ ]:
# 위 과정을 한번에 시행
df['sentence'] = df['검색필드'].map(lambda x:[e for e in x.split('。') if '</em>' in e])
df['sentence']

In [ ]:
df['sentence'][7449]

In [ ]:
# 불필요한 태그 삭제
#<em class="hl1">검색어</em> -> 검색어
df['sentence'] = df['sentence'].map(lambda x:[re.sub('\<em class\=\"hl1\"\>(.+)\<\/em\>','\g<1>',e) for e in x])
df['sentence']

In [ ]:
# 엑셀로 저장 (파일이름 : '검색어_DB이름')
df.to_excel(f'{keyword}_{secld}.xlsx',index=None)

### 기술통계량

In [ ]:
# 서명
df['서명'].value_counts()

In [ ]:
# 저자
df['저자'].value_counts()

In [ ]:
# 간행년
df['간행년'].value_counts()